In [1]:
from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

/home/matijak/Documents/programiranje/projects/semeval/merged_data/subtask1.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)
df.head()

,lang,art_name,entity,start,end,class1,classes2,text
0,BG,BG_670.txt,Запад,152,156,Antagonist,"[Conspirator, Instigator, Foreign Adversary]",Опитът на колективния Запад да „обезкърви Руси...
1,BG,BG_670.txt,САЩ,530,532,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
2,BG,BG_670.txt,НАТО,535,538,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
3,BG,BG_670.txt,Украйна,578,584,Antagonist,[Foreign Adversary],Опитът на колективния Запад да „обезкърви Руси...
4,BG,BG_670.txt,украински войници,633,649,Innocent,[Victim],Опитът на колективния Запад да „обезкърви Руси...


In [3]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)
df.loc[448]

lang                                                       EN
art_name                                     EN_UA_103861.txt
entity                                                Chinese
start                                                     791
end                                                       797
class1                                             Antagonist
classes2                                                [Spy]
text        The World Needs Peacemaker Trump Again \n\n by...
label                                                       0
input       The World Needs Peacemaker Trump Again  by Jef...
Name: 448, dtype: object

In [4]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)
    
    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]
    
    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
print(df.loc[448])

lang                                                            EN
art_name                                          EN_UA_103861.txt
entity                                                     Chinese
start                                                          791
end                                                            797
class1                                                  Antagonist
classes2                                                     [Spy]
text             The World Needs Peacemaker Trump Again \n\n by...
label                                                            0
input            The World Needs Peacemaker Trump Again  by Jef...
new_start_end                                           (785, 792)
Name: 448, dtype: object


In [5]:
import spacy

# Load the pre-trained models for each language
nlp_en = spacy.load('en_core_web_sm')  # English model
nlp_bg = spacy.load('xx_ent_wiki_sm')  # Bulgarian model
nlp_hi = spacy.load('xx_ent_wiki_sm')  # Hindi model
nlp_pt = spacy.load('xx_ent_wiki_sm')  # Portuguese model

nlp_en.add_pipe('sentencizer')
nlp_bg.add_pipe('sentencizer')
nlp_hi.add_pipe('sentencizer')
nlp_pt.add_pipe('sentencizer')


In [6]:
#ovo se vrti minutu i pol da se ne cudite
def sentencizer(row):
    sentences = 0
    if row['lang'] == 'EN':
        doc = nlp_en(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    if row['lang'] == 'HI':
        doc = nlp_hi(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    if row['lang'] == 'BG':
        doc = nlp_bg(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    if row['lang'] == 'PT':
        doc = nlp_pt(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    return sentences
df['sentences'] = df.apply(sentencizer,axis=1)
df.loc[0]

lang                                                            BG
art_name                                                BG_670.txt
entity                                                       Запад
start                                                          152
end                                                            156
class1                                                  Antagonist
classes2              [Conspirator, Instigator, Foreign Adversary]
text             Опитът на колективния Запад да „обезкърви Руси...
label                                                            0
input            Опитът на колективния Запад да „обезкърви Руси...
new_start_end                                           (151, 156)
sentences        [Опитът на колективния Запад да „обезкърви Рус...
Name: 0, dtype: object

In [7]:
def sent_id(row):
    sents = row['sentences']
    inp = row['input']
    start = row['new_start_end'][0]
    ind = 0
    ind_s = -1
    for s in range(len(sents)):
        for i in range(len(sents[s])):
            ind+=1
            if ind == start:
                ind_s = s
                break
        if ind_s > -1:
            break


    if ind_s >= len(sents):
        print("WTF",ind_s,len(sents))
    return ind_s

df['new_ind'] = df.apply(sent_id,axis=1)
print(df.loc[448])
nid = df['new_ind'].loc[448]
sent = df['sentences'].loc[448]
print(sent[nid])

lang                                                            EN
art_name                                          EN_UA_103861.txt
entity                                                     Chinese
start                                                          791
end                                                            797
class1                                                  Antagonist
classes2                                                     [Spy]
text             The World Needs Peacemaker Trump Again \n\n by...
label                                                            0
input            The World Needs Peacemaker Trump Again  by Jef...
new_start_end                                           (785, 792)
sentences        [The World Needs Peacemaker Trump Again  by Je...
new_ind                                                          5
Name: 448, dtype: object
TRUTH LIVES on at https://sgtreport.tv/ There has been an astounding 6,300% increase in the number of Ch

In [8]:
def sent_input(row):
    nind = row['new_ind']
    sents = row['sentences']
    inp = ""
    if nind - 1 >= 0:
        inp+=sents[nind-1]
    inp+=sents[nind]
    if nind + 1 < len(sents):
        inp+=sents[nind+1]
    return inp
def sent_input_short(row):
    nind = int(row['new_ind'])
    sents = row['sentences']
    inp = ""
    if nind >= len(sents) :
        print(nind,len(sents))
    inp+=sents[nind]
    return inp
df['context'] = df.apply(sent_input,axis=1)
df['no_context'] = df.apply(sent_input_short,axis=1)

print(df['context'].loc[448],
    df['no_context'].loc[448])

If this rate continues, this year’s total will easily top the 24,125 Chinese nationals who illegally entered the country last year.TRUTH LIVES on at https://sgtreport.tv/ There has been an astounding 6,300% increase in the number of Chinese nationals illegally entering the country in the last few years.Since China is a communist nation, these individuals are not freely traveling to the United States for a “better life.” TRUTH LIVES on at https://sgtreport.tv/ There has been an astounding 6,300% increase in the number of Chinese nationals illegally entering the country in the last few years.


In [9]:
from sklearn.model_selection import train_test_split
X = df["no_context"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=22
)
X_train.head(), X_test.head(), y_train.head(), y_test.head()

(814     Biden is a compromised puppet politician who h...
 937     मास्को और वॉशिंगटन के बीच संतुलन बनाना अमेरिका...
 728     The former head of German intelligence has rev...
 579     Speaking in the Estonian parliament on January...
 2392    O membro do alto escalão do Daesh (organização...
 Name: no_context, dtype: object,
 2109    O porta-voz do Kremlin (presidência russa), Dm...
 676     But his unconvincing 'no' will not stop Ayatol...
 946     इस संचार युद्ध में रूस कटघरे में खड़ा हो चुका है।
 1       Подкрепата на САЩ, НАТО и други западни съюзни...
 1697    यूक्रेन के रूसी आक्रमण के 42 वें दिन जारी की ग...
 Name: no_context, dtype: object,
 814     0
 937     0
 728     0
 579     2
 2392    0
 Name: label, dtype: int64,
 2109    0
 676     0
 946     2
 1       0
 1697    1
 Name: label, dtype: int64)

In [10]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [11]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [21]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from datasets import Dataset

# Convert pandas DataFrame/Series to Hugging Face Dataset
train_data = pd.DataFrame({'text': X_train, 'label': y_train})
test_data = pd.DataFrame({'text': X_test, 'label': y_test})

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [23]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2154 [00:00<?, ? examples/s]

Map:   0%|          | 0/381 [00:00<?, ? examples/s]

In [24]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
print(train_dataset)

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2154
})


In [25]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64)
optimizer = AdamW(model.parameters(), lr=2e-5)

In [26]:
num_epochs = 5
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    # Initialize tqdm progress bar for training
    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")
    
    for batch in train_progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=-1)
        #print(preds)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)
        
        # Update tqdm description with current loss
        train_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
    
    # Test phase
    model.eval()
    test_loss = 0
    correct_test_predictions = 0
    total_test_predictions = 0
    
    # Initialize tqdm progress bar for test
    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")
    
    with torch.no_grad():
        for batch in test_progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            test_loss += loss.item()
            
            # Calculate test accuracy
            preds = torch.argmax(logits, dim=-1)
            correct_test_predictions += (preds == labels).sum().item()
            total_test_predictions += labels.size(0)
            
            # Update tqdm description with current loss
            test_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions
    
    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

Training Epoch 1/5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:27<00:00,  1.22it/s, loss=1.06]


Epoch 1/5
Training loss: 1.0441, Training accuracy: 0.4777


Test Epoch 1/5: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.90it/s, loss=0.996]


Test loss: 1.0120, Test accuracy: 0.5039


Training Epoch 2/5: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:27<00:00,  1.21it/s, loss=0.976]


...

Test loss: 0.8306, Test accuracy: 0.6273


Training Epoch 5/5: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:27<00:00,  1.22it/s, loss=0.702]


Epoch 5/5
Training loss: 0.6580, Training accuracy: 0.7405


Test Epoch 5/5: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.97it/s, loss=0.704]

Test loss: 0.8012, Test accuracy: 0.6745
